In [1]:
import numpy as np
import pandas as pd
import sqlite3


# 1. 数组组合-concat函数

In [ ]:
df1 = pd.read_csv('mdata/concat_1.csv')
df2 = pd.read_csv('mdata/concat_2.csv')
df3 = pd.read_csv('mdata/concat_3.csv')
df1
df2
df3

In [ ]:
# concat函数，把df对象连接起来
pd.concat([df1,df2,df3])
pd.concat([df1,df2,df3],axis='rows')
pd.concat([df1,df2,df3],axis=0) # 按行拼接
pd.concat([df1,df2,df3],axis= 1 ) # 按列拼接

In [ ]:
# 把DataFrame和Series对象拼接到一起
# 由于series是列数据，concat方法默认是添加行，但是series没有行所以，所以添加了新的列，缺失值用nan填充
s1 = pd.Series(['n1','n2','n3'])
pd.concat([df1,s1])

In [ ]:
# 将['n1','n2','n3','n4']作为行连接到df1之后
# 按行拼接，参考列名；按列拼接，参考行名
df5 = pd.DataFrame([['n1','n2','n3','n4']],columns=['A','B','C','D']) #这里得套两层，套一层识别为series
pd.concat([df1,df5],ignore_index=True)

In [ ]:
pd.concat([df1,df2],axis='columns')
pd.concat([df1,df2],axis=1)

In [ ]:
df1['new_col1'] = 'ai20'
df1

df1['new_col2'] = ['张三','李四','王五','赵六']
df1

# 还可以吧添加的列值封装为series
df1['new_col2'] = pd.Series(['hh','aa','zz','xx'])
df1

# 2.merge()方式  合并数据

## 2.1 一对一合并

In [ ]:
# 从sqlite中读取数据
conn = sqlite3.connect('mdata/chinook.db')
tracks = pd.read_sql_query('select * from tracks;' ,conn) # 歌曲表
tracks.head()

genres = pd.read_sql_query('select * from genres;',conn)
genres.head()

In [ ]:
# 2. 为了更好的演示，连接查询，防止不相关的列干扰我们。我们从歌曲表中抽取一些数据
tracks_subset = tracks.loc[[0,62,76,98,110,193,204,281,322,359]]
tracks_subset



In [ ]:
# 通过merge函数，实现tracks_subset和genres连接操作
# 格式 dfa.merge(dfb,on='关联字段',how='连接方式')
# 如果两个df对象关联字段一样，用on直接连接，如果不一样，用left_on = '左df字段名'，right_on = '右df字段名'
genres.merge(tracks_subset[['TrackId','GenreId','Milliseconds']],on = 'GenreId',how = 'left')
genres.merge(tracks_subset[['TrackId','GenreId','Milliseconds']],on = 'GenreId',how = 'right')
genres.merge(tracks_subset[['TrackId','GenreId','Milliseconds']],on = 'GenreId',how = 'outer')


In [ ]:
# 如果两个df的字段重名了，则suffixes=('_x','_y')会分别给左df 和 右df加后缀，以示区分
genres.merge(tracks_subset[['TrackId','Name','GenreId','Milliseconds']],on = 'GenreId',how = 'outer')


## 2.2 多对一合并

In [ ]:
genres.merge(tracks_subset) # 一对一，因为tracks_subset中只有10条数据，且歌曲分类id都是不同的
genres.merge(tracks) # 一对多，tracks中多收歌曲可能属于一个类别
genre_track =genres.merge(tracks[['TrackId','GenreId','Milliseconds']],on = 'GenreId', how = 'left')
genre_track 

# 基于上述数据，按照歌曲类别分组，计算平均时长
tmp_series = genre_track.groupby(['GenreId','Name'])['Milliseconds'].mean()
tmp_series

## 基于上述数据，构成日期格式
pd.to_timedelta(tmp_series,unit='ms').dt.floor('s').sort_values()